# 1. Class tensor

## 1.1. tf.Tensor()

Là những kí tự hoặc tên gọi thể hiện kết quả đầu ra của một operation. Về bản chất nó chính là các object được khởi tạo từ các operation. Chẳng hạn như chúng ta có `c,d` trong triển khai bên dưới được gọi là tensor.

In [57]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])



## 1.2. tf.SparseTensor()

Là một dạng Tensor đặc biệt chỉ quan tâm đến các giá trị khác 0 của một tensor thông thường và tìm các biểu diễn tensor theo một dạng mới mà chỉ bao gồm các giá trị khác 0 và chỉ số vị trí của nó. Tensorflow biểu diễn SparseTensor thông qua 3 thành phần: `indices, values, dense_shape`. Do đó nó có cú pháp `SparseTensor(indices, values, dense_shape)`. Giả sử N và n_dims lần lượt là số quan sát và số chiều của SparseTensor.

* indices:  Là một 2D-tensor của tensor `dense_shape` có giá trị đại diện cho `[vị trí quan sát, vị trí chiều]`. Xác định vị trí của của các phần tử khác 0 của SparseTensor. Chẳng hạn indices = [[1, 2], [3, 4]] thì các phần tử có vị trí [1, 2] và [3, 4] khác 0.
* values: Là 1D-tensor xác định các giá trị của từng phần tử tương ứng với vị trí được xác định trong indices. Chẳng hạn với values = [32, 11] thì các phần tử ở vị trí [1, 2] bằng 32 và vị trí [3, 4] bằng 11.
* dense_shape: Là 1D-tensor qui định các chiều của sparse tensor. Chẳng hạn dense_shape = [5, 4] ta hiểu rằng sparse tensor có số quan sát N = 5 và số chiều n_dims = 4.

Bên dưới là ví dụ của SparseTensor.

In [55]:
import tensorflow as tf
x = tf.SparseTensor(indices = [[1, 0], [1, 2]], values = [2.1, 3.2], dense_shape = [4, 4])
with tf.Session() as sess:
    print(sess.run(x))

SparseTensorValue(indices=array([[1, 0],
       [1, 2]], dtype=int64), values=array([2.1, 3.2], dtype=float32), dense_shape=array([4, 4], dtype=int64))


Biểu diễn dưới dạng tensor thông thường như sau:

In [56]:
y = tf.sparse_tensor_to_dense(x)
with tf.Session() as sess:
    print(sess.run(y))

[[0.  0.  0.  0. ]
 [2.1 0.  3.2 0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


# 2. tf.data.Dataset()

Khi chúng ta xây dựng một mạng nơ ron chúng ta sẽ cần quản lý quá trình truyền dữ liệu đầu vào theo các batch. Để làm được việc đó thì chúng ta sẽ sử dụng class Dataset. 
Class Dataset được dùng để biểu diễn một phương thức truyền dữ liệu đầu vào của một tợp hợp tensors (các tensor của biến dự báo và tensor của biến mục tiêu) và những phương thức biến đổi dữ liệu chứa trong những thành phần này.
Kết quả trả về của Class Dataset là một Tensor hoặc Sparse Tensor.

Cách khởi tạo một object Dataset gồm:

## 2.1. Tạo Dataset từ hàm khởi tạo

Chúng ta sẽ sử dụng một hàm số để khởi tạo để tạo ra Dataset thông qua hàm `Dataset.from_generator()` có cú pháp như sau:

`
from_generator(
    generator,
    output_types,
    output_shapes=None,
    args=None
)
`

Trong đó generator là hàm khởi tạo sao cho nó không trả về kết quả mà trả về một hàm ở trạng thái pending (kết quả trả về của hàm được để sau yield thay vì return), output_types là kiểu biến, output_shapes là hình dạng dữ liệu.

In [86]:
import itertools
import tensorflow as tf
# Khởi tạo một generator.
def gen():
    for i in itertools.count(1):
        yield (i, [1]*i)
        
ds = tf.data.Dataset.from_generator(
    generator = gen, 
    output_types = (tf.int64, tf.int64),
    output_shapes = (tf.TensorShape([]), tf.TensorShape([None]))
)

value = ds.make_one_shot_iterator().get_next()

sess = tf.InteractiveSession()
sess.run(value)

c:\users\laptoptcc\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(1, array([1], dtype=int64))

In [81]:
sess.run(value)

(2, array([1, 1], dtype=int64))

Hàm make_one_shot_iterator() sẽ khởi tạo một enumerate để truy cập từng phần tử trong dataset. Do generator đang ở trạng thái pending nên mỗi lần gọi sess thì nó sẽ kích hoạt giá trị đầu tiên được trả về. Do đó giá trị của record tiếp theo chính là kết quả trả về của vòng lặp trong gen khi i tăng thêm 1 đơn vị so với trước đó.

## 2.2. Tạo Dataset từ một tensor

Chúng ta có thể tạo ra một Dataset từ một tensor bằng hàm `from_tensors(tensors)`.

In [93]:
import numpy as np
ds = tf.data.Dataset.from_tensors(tf.constant(np.arange(10).reshape(-1, 2)))
#Khởi tạo một iterator để truy cập các thành phần của dataset
value = ds.make_one_shot_iterator().get_next()
sess.run(value)

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

Hoặc ta có thể sử dụng hàm `from_tensor_slices(tensors)` để khởi tạo Dataset. Khi đó Dataset là tợp hợp các lát cắt của tensors. 

In [95]:
ds = tf.data.Dataset.from_tensor_slices(tf.constant(np.arange(10).reshape(-1, 2)))
value = ds.make_one_shot_iterator().get_next()
sess.run(value)

array([0, 1])

## 2.3. Map một function vào Dataset

Chúng ta cần biến đổi các giá trị của Dataset theo một hàm nào đó. Khi đó sử dụng `interleave()` sẽ cho phép chúng ta áp dụng hàm số vào Dataset một cách xen kẽ. Cú pháp:

`
interleave(
    map_func,
    cycle_length,
    block_length=1
)
`

cycle_length và block_length sẽ lần lượt kiểm soát số lượng phần tử đầu vào và số lượng các các phần tử trong 1 vòng lặp được đưa vào. Bên dưới là ví dụ của việc map một hàm số vào 1 dataset. 

In [123]:
# NOTE: The following examples use `{ ... }` to represent the
# contents of a dataset.
a = tf.data.Dataset.from_tensors(tf.constant(np.arange(10)))

# NOTE: New lines indicate "block" boundaries.
a.interleave(lambda x: tf.data.Dataset.from_tensors(x).repeat(6),
             cycle_length=2, block_length=4) == {
    1, 1, 1, 1,
    2, 2, 2, 2,
    1, 1,
    2, 2,
    3, 3, 3, 3,
    4, 4, 4, 4,
    3, 3,
    4, 4,
    5, 5, 5, 5,
    5, 5,
}

False

In [124]:
value = a.make_one_shot_iterator().get_next()
sess.run(value)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Để biến đổi các giá trị của một Dataset theo một hàm số nào đó, chúng ta sử dụng hàm map như sau:
`
map(
    map_func,
    num_parallel_calls=None
)
`
trong đó map_func là hàm biến đổi.

In [149]:
# NOTE: The following examples use `{ ... }` to represent the
# contents of a dataset.
a = tf.data.Dataset.from_tensors(tf.constant(np.arange(1, 6)))
sess.run(a.map(lambda x: x**2).make_one_shot_iterator().get_next())

array([ 1,  4,  9, 16, 25])

In [152]:
a = tf.data.Dataset.from_tensors((tf.constant(np.arange(1, 4)), tf.constant(["foo", "bar", "zoo"])))
sess.run(a.make_one_shot_iterator().get_next())

(array([1, 2, 3]), array([b'foo', b'bar', b'zoo'], dtype=object))

In [157]:
ds = tf.data.TextLineDataset('iris_training.csv')
sess.run(ds.make_one_shot_iterator().get_next())

b'120,4,setosa,versicolor,virginica'

In [159]:
ds = tf.data.Dataset.from_tensors('iris_training.csv')
sess.run(ds.make_one_shot_iterator().get_next())

b'iris_training.csv'